detectron2 using cocolike structure training mask rcnn for 5000 iterations.

In [1]:
import torch, torchvision
print(torch.__version__, torch.cuda.is_available(), torch.version.cuda)

1.6.0 True 10.1


In [2]:
import os
import copy
import json
import pycocotools
import random 

import matplotlib.pyplot as plt
from tqdm import tqdm
from pathlib import Path
from collections import defaultdict

assert torch.__version__.startswith("1.6")

import detectron2
import detectron2.data.transforms as T
import detectron2.utils.comm as comm

from detectron2.utils.logger import setup_logger
setup_logger()

from detectron2 import model_zoo
from detectron2.engine import DefaultTrainer, DefaultPredictor
from detectron2.config import get_cfg

from detectron2.data import MetadataCatalog,DatasetMapper,build_detection_train_loader,build_detection_test_loader
from detectron2.data import detection_utils as utils
from detectron2.data.catalog import DatasetCatalog
from detectron2.data.datasets import register_coco_instances 

from detectron2.evaluation import COCOEvaluator, inference_on_dataset

from detectron2.projects.deeplab import add_deeplab_config, build_lr_scheduler

In [3]:
PATH = os.path.abspath(os.getcwd())

register_coco_instances("my_dataset_train_v1",{},PATH + "/input/train_annotations.json",PATH + "/input/train_v2/")
register_coco_instances("my_dataset_val_v1",{},PATH + "/input/test_annotations.json",PATH + "/input/train_v2/")

my_dataset_train_metadata = MetadataCatalog.get("my_dataset_train_v1")
dataset_dicts = DatasetCatalog.get("my_dataset_train_v1")

[11/28 14:21:47 d2.data.datasets.coco]: Loaded 38301 images in COCO format from /application/input/train_annotations.json


In [8]:
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_X_101_32x8d_FPN_3x.yaml"))

cfg.DATASETS.TRAIN = ("my_dataset_train_v1",) 
cfg.DATASETS.TEST = ("my_dataset_val_v1",)
cfg.TEST.EVAL_PERIOD = 500 ## // TODO useful to set up eval_hook?
cfg.DATALOADER.NUM_WORKERS = 4 ## 4 per gpu
cfg.SOLVER.IMS_PER_BATCH = 20 
cfg.SOLVER.BASE_LR = 0.001  # pick a good LR
cfg.SOLVER.MAX_ITER = 10000
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 512 
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1  # only has one class (ship)

In [5]:
def custom_mapper(dataset_dict):
    dataset_dict = copy.deepcopy(dataset_dict)  # it will be modified by code below
    image = utils.read_image(dataset_dict["file_name"], format="BGR")
    # List of transforms https://detectron2.readthedocs.io/modules/data_transforms.html
    # Add saturation, add shear orsmth.
    transform_list = [T.RandomCrop("absolute",(256,256)), 
                      T.RandomFlip(prob=0.5, horizontal=False, vertical=True),
                      T.RandomFlip(prob=0.5, horizontal=True, vertical=False),
                     ]
    image, transforms = T.apply_transform_gens(transform_list, image)
    dataset_dict["image"] = torch.as_tensor(image.transpose(2, 0, 1).astype("float32"))

    annos = [
        utils.transform_instance_annotations(obj, transforms, image.shape[:2])
        for obj in dataset_dict.pop("annotations")
        if obj.get("iscrowd", 0) == 0
    ]
    instances = utils.annotations_to_instances(annos, image.shape[:2])
    dataset_dict["instances"] = utils.filter_empty_instances(instances)
    return dataset_dict


from detectron2.engine import DefaultTrainer
from detectron2.evaluation import COCOEvaluator
class CocoTrainer(DefaultTrainer):
    @classmethod
    def build_evaluator(cls, cfg, dataset_name, output_folder=None):
        if output_folder is None:
            os.makedirs("coco_eval", exist_ok=True)
            output_folder = "coco_eval"
        return COCOEvaluator(dataset_name, cfg, False, output_folder)
    
    @classmethod
    def build_train_loader(cls, cfg):
        return build_detection_train_loader(cfg, mapper=custom_mapper)
    
    @classmethod
    def build_lr_scheduler(cls, cfg, optimizer):
        return build_lr_scheduler(cfg, optimizer)


In [8]:
# //TODO make tensorboard work.

%load_ext tensorboard
%tensorboard --logdir=output

from tensorboard import notebook
notebook.list()
notebook.display(port=6006, height=1000) 

Known TensorBoard instances:
  - port 6006: logdir output (started 0:00:01 ago; pid 46)
Selecting TensorBoard with logdir output (started 0:00:01 ago; port 6006, pid 46).


In [9]:
os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
trainer = CocoTrainer(cfg) 
trainer.resume_or_load(resume=True) #True takes last checkpoint file which is saved below.
trainer.train() #Trainer will throw out non-annotated pictures. 

[11/28 14:47:16 d2.engine.defaults]: Model:
GeneralizedRCNN(
  (backbone): FPN(
    (fpn_lateral2): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral3): Conv2d(512, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral4): Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral5): Conv2d(2048, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output5): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (top_block): LastLevelMaxPool()
    (bottom_up): ResNet(
      (stem): BasicStem(
        (conv1): Conv2d(
          3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False
          (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
        )
      )
      (res

[11/28 14:47:17 d2.data.datasets.coco]: Loading /application/input/train_annotations.json takes 1.04 seconds.
[11/28 14:47:17 d2.data.datasets.coco]: Loaded 38301 images in COCO format from /application/input/train_annotations.json
[11/28 14:47:18 d2.data.build]: Removed 0 images with no usable annotations. 38301 images left.
[11/28 14:47:20 d2.data.build]: Using training sampler TrainingSampler
[11/28 14:47:20 d2.data.common]: Serializing 38301 elements to byte tensors and concatenating them all ...
[11/28 14:47:20 d2.data.common]: Serialized dataset takes 19.64 MiB
[11/28 14:47:21 d2.engine.train_loop]: Starting training from iteration 5250
[11/28 14:47:31 d2.utils.events]:  eta: 0:03:53  iter: 5259  total_loss: 0.3442  loss_cls: 0.01505  loss_box_reg: 0.02022  loss_mask: 0.2877  loss_rpn_cls: 0.009746  loss_rpn_loc: 0.009597  time: 0.9724  data_time: 0.1330  lr: 0.0025  max_mem: 9122M
[11/28 14:47:51 d2.utils.events]:  eta: 0:03:34  iter: 5279  total_loss: 0.3132  loss_cls: 0.01094 

[11/28 14:54:36 d2.evaluation.evaluator]: Inference done 1287/4255. 0.1233 s / img. ETA=0:06:52
[11/28 14:54:41 d2.evaluation.evaluator]: Inference done 1325/4255. 0.1234 s / img. ETA=0:06:46
[11/28 14:54:46 d2.evaluation.evaluator]: Inference done 1360/4255. 0.1235 s / img. ETA=0:06:41
[11/28 14:54:51 d2.evaluation.evaluator]: Inference done 1394/4255. 0.1237 s / img. ETA=0:06:37
[11/28 14:54:56 d2.evaluation.evaluator]: Inference done 1429/4255. 0.1238 s / img. ETA=0:06:33
[11/28 14:55:01 d2.evaluation.evaluator]: Inference done 1466/4255. 0.1238 s / img. ETA=0:06:28
[11/28 14:55:06 d2.evaluation.evaluator]: Inference done 1503/4255. 0.1238 s / img. ETA=0:06:22
[11/28 14:55:11 d2.evaluation.evaluator]: Inference done 1540/4255. 0.1238 s / img. ETA=0:06:17
[11/28 14:55:16 d2.evaluation.evaluator]: Inference done 1576/4255. 0.1238 s / img. ETA=0:06:12
[11/28 14:55:21 d2.evaluation.evaluator]: Inference done 1614/4255. 0.1238 s / img. ETA=0:06:07
[11/28 14:55:26 d2.evaluation.evaluator]

[11/28 15:01:16 d2.evaluation.evaluator]: Inference done 4174/4255. 0.1239 s / img. ETA=0:00:11
[11/28 15:01:21 d2.evaluation.evaluator]: Inference done 4211/4255. 0.1239 s / img. ETA=0:00:06
[11/28 15:01:26 d2.evaluation.evaluator]: Inference done 4248/4255. 0.1239 s / img. ETA=0:00:00
[11/28 15:01:27 d2.evaluation.evaluator]: Total inference time: 0:09:49.516363 (0.138710 s / img per device, on 1 devices)
[11/28 15:01:27 d2.evaluation.evaluator]: Total inference pure compute time: 0:08:46 (0.123935 s / img per device, on 1 devices)
[11/28 15:01:28 d2.evaluation.coco_evaluation]: Preparing results for COCO format ...
[11/28 15:01:28 d2.evaluation.coco_evaluation]: Saving results to coco_eval/coco_instances_results.json
[11/28 15:01:28 d2.evaluation.coco_evaluation]: Evaluating predictions with unofficial COCO API...
Loading and preparing results...
DONE (t=0.07s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
COCOeval_opt.evaluate() fi

[11/28 15:03:37 d2.evaluation.evaluator]: Inference done 880/4255. 0.1259 s / img. ETA=0:07:51
[11/28 15:03:42 d2.evaluation.evaluator]: Inference done 915/4255. 0.1261 s / img. ETA=0:07:47
[11/28 15:03:47 d2.evaluation.evaluator]: Inference done 952/4255. 0.1260 s / img. ETA=0:07:41
[11/28 15:03:52 d2.evaluation.evaluator]: Inference done 987/4255. 0.1262 s / img. ETA=0:07:37
[11/28 15:03:57 d2.evaluation.evaluator]: Inference done 1022/4255. 0.1263 s / img. ETA=0:07:32
[11/28 15:04:02 d2.evaluation.evaluator]: Inference done 1059/4255. 0.1263 s / img. ETA=0:07:27
[11/28 15:04:07 d2.evaluation.evaluator]: Inference done 1096/4255. 0.1262 s / img. ETA=0:07:21
[11/28 15:04:12 d2.evaluation.evaluator]: Inference done 1133/4255. 0.1261 s / img. ETA=0:07:16
[11/28 15:04:17 d2.evaluation.evaluator]: Inference done 1170/4255. 0.1260 s / img. ETA=0:07:10
[11/28 15:04:22 d2.evaluation.evaluator]: Inference done 1206/4255. 0.1262 s / img. ETA=0:07:05
[11/28 15:04:27 d2.evaluation.evaluator]: In

[11/28 15:10:17 d2.evaluation.evaluator]: Inference done 3773/4255. 0.1253 s / img. ETA=0:01:06
[11/28 15:10:22 d2.evaluation.evaluator]: Inference done 3809/4255. 0.1253 s / img. ETA=0:01:01
[11/28 15:10:27 d2.evaluation.evaluator]: Inference done 3845/4255. 0.1253 s / img. ETA=0:00:56
[11/28 15:10:32 d2.evaluation.evaluator]: Inference done 3881/4255. 0.1253 s / img. ETA=0:00:51
[11/28 15:10:37 d2.evaluation.evaluator]: Inference done 3917/4255. 0.1253 s / img. ETA=0:00:46
[11/28 15:10:42 d2.evaluation.evaluator]: Inference done 3953/4255. 0.1253 s / img. ETA=0:00:41
[11/28 15:10:47 d2.evaluation.evaluator]: Inference done 3991/4255. 0.1253 s / img. ETA=0:00:36
[11/28 15:10:52 d2.evaluation.evaluator]: Inference done 4027/4255. 0.1253 s / img. ETA=0:00:31
[11/28 15:10:57 d2.evaluation.evaluator]: Inference done 4064/4255. 0.1253 s / img. ETA=0:00:26
[11/28 15:11:02 d2.evaluation.evaluator]: Inference done 4101/4255. 0.1253 s / img. ETA=0:00:21
[11/28 15:11:07 d2.evaluation.evaluator]

In [8]:
## save the training.
from detectron2.checkpoint import DetectionCheckpointer

checkpointer = DetectionCheckpointer(trainer.model, save_dir="./")
checkpointer.save("ship_ml")  # save to save_dir

NameError: name 'DetectionCheckpointer' is not defined